## Demo of geopandas:
### Calculate total quarter-mile service area of all heavy-rail Red Line stations

In [ ]:
import geopandas as gp

In [ ]:
def make_gdf_from_wfs_request(base_url, typename_parm):
    request_url = base_url + '?' + 'service=wfs'
    request_url += '&request=getfeature'
    request_url += '&typename=' + typename_parm
    request_url += '&outputformat=json'
    gdf = gp.read_file(request_url)
    return gdf

In [ ]:
# MassGIS WFS base URL
massgis_base_url = 'https://gis-prod.digital.mass.gov/geoserver/wfs'

In [ ]:
stations_gdf = make_gdf_from_wfs_request(massgis_base_url, 'massgis:GISDATA.MBTA_NODE')

In [ ]:
stations_gdf.plot()

In [ ]:
stations_gdf.explore()

### Aside: The read_file method is polymorphic. You don't have to specify the format of the data to be loaded!

In [ ]:
# Let's load a shapefile...
rpas_gdf = gp.read_file("data/RPAS_POLY.shp")

In [ ]:
rpas_gdf.explore()

### Back to our main topic...
### Let's take a look at the Red Line stations in our geodataframe.

In [ ]:
stations_gdf[stations_gdf['line'] == 'RED']

### What happened to Park Street (and Downtown Crossing)?

In [ ]:
stations_gdf[stations_gdf['station'].str.contains('Park')]

### We need to refine our query:
##### 1. A more effective filter on 'line'
##### 2. Filter out Mattapan Trolley (light rail) stations

In [ ]:
# A more effective filter on 'line'
temp_gdf = stations_gdf[stations_gdf['line'].str.contains('RED')]
temp_gdf

In [ ]:
# Filter out Mattapan Trolley stations
temp_gdf[~temp_gdf['route'].str.contains('Mattapan')]

#### We can combine the two parts of the query into a single query

In [ ]:
stations_red = stations_gdf[(stations_gdf['line'].str.contains('RED')) & (~stations_gdf['route'].str.contains('Mattapan'))]

In [ ]:
stations_red.plot()

In [ ]:
# Calculate the quarter-mile service area of each staion
quarter_mile_in_meters = 0.25 *  1609.344
buf = stations_red.buffer(quarter_mile_in_meters)

In [ ]:
buf.head()

In [ ]:
# The 'buffer' method returns a GeoSeries; turn it into a GeoDataFrame.
buf_gdf = gp.GeoDataFrame(gp.GeoSeries(buf))
buf_gdf = buf_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')

In [ ]:
buf_gdf.head()

In [ ]:
buf_gdf.explore()

In [ ]:
# Dissolve buffers
buf_gdf = buf_gdf.dissolve()

In [ ]:
buf_gdf.explore()

#### A service area over open water shouldn't be counted

In [ ]:
# Create a polygon feature to clip out all "open water"
towns_gdf = make_gdf_from_wfs_request(massgis_base_url, 'massgis:GISDATA.TOWNSSURVEY_POLY')
towns_gdf = towns_gdf[towns_gdf['coastal_poly'] == 'NO']
clipping_gdf = towns_gdf.dissolve()

In [ ]:
buf_gdf_clipped = buf_gdf.clip(clipping_gdf)

In [ ]:
buf_gdf_clipped.explore()

In [ ]:
# Get total of all quarter-mile service areas, in square miles
# Recall: SRS of data uses 'meters' as unit of measure
SQ_METERS_TO_SQ_MILES_FACTOR = 0.00000038610
buf_gdf_clipped['area_sq_mi'] = buf_gdf_clipped.area * SQ_METERS_TO_SQ_MILES_FACTOR

In [ ]:
buf_gdf_clipped